# La Corrección de Bonferroni: Controlando el Error Familiar 🛡️

---

## **# 1. El Problema: Inflación del Error Tipo I ($\alpha$)**

En la estadística de series de tiempo y en la ciencia de datos en general, a menudo necesitamos realizar **múltiples pruebas de hipótesis** simultáneamente. Por ejemplo:
* Correr la **prueba de Ljung-Box** para la no autocorrelación de residuos en 10, 20 o 30 *lags* diferentes.
* Comparar la media de ventas entre 50 tiendas distintas (50 comparaciones).
* Probar la significancia de 100 *features* diferentes en un modelo.

### **El Riesgo del Error Tipo I (Falso Positivo)**

Recordemos que el **Error Tipo I** ($\alpha$) es la probabilidad de **rechazar la Hipótesis Nula ($H_0$) cuando en realidad es verdadera**. Generalmente fijamos este nivel en $\alpha = 0.05$ (o 5%).

Cuando realizamos **solo una prueba**, la probabilidad de cometer un falso positivo es del 5%.

Cuando realizamos **$m$ pruebas independientes**, la probabilidad de cometer *al menos un* Falso Positivo se dispara. Esto se conoce como la **Tasa de Error Familiar** (*Family-Wise Error Rate*, FWER).

### **Fórmula del Error Familiar (FWER)**

Si realizamos $m$ pruebas con un nivel de significancia individual $\alpha$, la probabilidad de cometer **al menos un Error Tipo I** es:

$$
\text{FWER} = 1 - (1 - \alpha)^m
$$

| Número de Pruebas ($m$) | $\alpha = 0.05$ | FWER (Riesgo real) |
| :--- | :--- | :--- |
| 1 | 0.05 | 5.0% |
| 5 | 0.05 | 22.6% |
| 20 | 0.05 | **64.2%** |

**Conclusión:** Si realizamos 20 pruebas de Ljung-Box con $\alpha=0.05$, tenemos más del 64% de probabilidad de declarar erróneamente que **al menos uno** de esos *lags* es significativo, incluso si no lo es (un falso positivo).

---

## **# 2. La Corrección de Bonferroni**

La Corrección de Bonferroni es el método más simple y conservador para controlar la Tasa de Error Familiar (FWER).

El método ajusta el nivel de significancia individual ($\alpha_{individual}$) para garantizar que la FWER total de todo el "experimento familiar" sea menor o igual al nivel de significancia deseado (ej. 0.05).

### **Fórmula de Bonferroni (El Nivel $\alpha$ Ajustado)**

Para mantener una FWER total de $\alpha_{deseado}$ al realizar $m$ pruebas, el nuevo nivel de significancia para **cada prueba individual** es:

$$
\alpha_{\text{Bonferroni}} = \frac{\alpha_{\text{deseado}}}{m}
$$

Donde:
* $\alpha_{\text{deseado}}$: El nivel de significancia que quieres para todo el experimento (ej. 0.05).
* $m$: El número total de pruebas realizadas.
* $\alpha_{\text{Bonferroni}}$: El nuevo *p-valor umbral* que deben superar tus resultados.

### **Ejemplo Numérico**

Si quieres que tu **FWER** sea 0.05 y realizas $m=20$ pruebas:

$$
\alpha_{\text{Bonferroni}} = \frac{0.05}{20} = 0.0025
$$

**Regla de Decisión Ajustada:**
* Solo rechazarás la Hipótesis Nula ($H_0$) de una prueba si su p-valor es **menor que $0.0025$** (en lugar de $0.05$).

---

## **# 3. Importancia y Limitaciones**

### **Importancia (Por qué la usamos)**

* **Control del FWER:** Garantiza que la probabilidad de encontrar *algún* falso positivo en tu conjunto de pruebas se mantenga en el nivel deseado (ej. 5%).
* **Simplicidad:** Es muy fácil de calcular e implementar.
* **Robustez:** Es universalmente aplicable, independientemente de si las pruebas son dependientes o independientes.

### **Limitaciones (El Trade-off del Científico de Datos)**

La Corrección de Bonferroni es famosa por ser **demasiado conservadora**.

1.  **Aumento del Error Tipo II ($\beta$):** Al reducir drásticamente el umbral $\alpha$ (de 0.05 a 0.0025, por ejemplo), se hace mucho más difícil rechazar $H_0$. Esto aumenta la probabilidad de cometer un **Error Tipo II** (*False Negative*): **aceptar $H_0$ cuando en realidad es falsa** (perderte un patrón o una diferencia real).
2.  **Modelos Alternativos:** Debido a esta conservatividad, en la práctica profesional a menudo se prefieren métodos menos estrictos pero más potentes (que tienen un mejor equilibrio entre el Error Tipo I y el Tipo II), como el método de **Benjamini-Hochberg (BH)**, que controla la **Tasa de Descubrimiento Falso** (*False Discovery Rate*, FDR) en lugar del FWER.

In [1]:
import numpy as np

# 1. Parámetros del experimento
alfa_deseado = 0.05  # Nivel de significancia del 5% para todo el 'experimento'
m_pruebas = 10       # Número de lags que probamos en el Ljung-Box

# 2. Calcular el umbral de Bonferroni
alfa_bonferroni = alfa_deseado / m_pruebas

print(f"Número de pruebas (m): {m_pruebas}")
print(f"Nivel de significancia ajustado (alfa_bonferroni): {alfa_bonferroni:.4f}")

# 3. Datos de ejemplo: p-valores de 10 pruebas de Ljung-Box (simulados)
p_valores_simulados = np.array([0.045, 0.003, 0.001, 0.150, 0.002, 0.060, 0.048, 0.0005, 0.200, 0.300])

print("\n--- Resultados ---")

for i, p_val in enumerate(p_valores_simulados):
    es_significativo_bonf = p_val < alfa_bonferroni
    es_significativo_normal = p_val < alfa_deseado
    
    # Decisión normal (riesgo alto de Falso Positivo)
    decision_normal = "RECHAZA H0 (Significativo)" if es_significativo_normal else "NO RECHAZA H0"
    
    # Decisión Bonferroni (controlando FWER)
    decision_bonf = "RECHAZA H0 (Significativo)" if es_significativo_bonf else "NO RECHAZA H0"
    
    print(f"Lag {i+1}: p-valor={p_val:.4f}")
    print(f"  > Normal (0.05): {decision_normal}")
    print(f"  > Bonferroni ({alfa_bonferroni:.4f}): {decision_bonf}")

# RESUMEN DEL EFECTO
print("\n--- Efecto del Ajuste ---")
print(f"Total de significativos (Normal): {sum(p_valores_simulados < alfa_deseado)}")
print(f"Total de significativos (Bonferroni): {sum(p_valores_simulados < alfa_bonferroni)}")

Número de pruebas (m): 10
Nivel de significancia ajustado (alfa_bonferroni): 0.0050

--- Resultados ---
Lag 1: p-valor=0.0450
  > Normal (0.05): RECHAZA H0 (Significativo)
  > Bonferroni (0.0050): NO RECHAZA H0
Lag 2: p-valor=0.0030
  > Normal (0.05): RECHAZA H0 (Significativo)
  > Bonferroni (0.0050): RECHAZA H0 (Significativo)
Lag 3: p-valor=0.0010
  > Normal (0.05): RECHAZA H0 (Significativo)
  > Bonferroni (0.0050): RECHAZA H0 (Significativo)
Lag 4: p-valor=0.1500
  > Normal (0.05): NO RECHAZA H0
  > Bonferroni (0.0050): NO RECHAZA H0
Lag 5: p-valor=0.0020
  > Normal (0.05): RECHAZA H0 (Significativo)
  > Bonferroni (0.0050): RECHAZA H0 (Significativo)
Lag 6: p-valor=0.0600
  > Normal (0.05): NO RECHAZA H0
  > Bonferroni (0.0050): NO RECHAZA H0
Lag 7: p-valor=0.0480
  > Normal (0.05): RECHAZA H0 (Significativo)
  > Bonferroni (0.0050): NO RECHAZA H0
Lag 8: p-valor=0.0005
  > Normal (0.05): RECHAZA H0 (Significativo)
  > Bonferroni (0.0050): RECHAZA H0 (Significativo)
Lag 9: p-valor